In [1]:
# Imports here
import numpy as np
import json

In [4]:
# Dataset here
file = open("./dataset/processed_reviews.json", 'r', encoding='utf8')
dataset_dict = json.load(file)

In [5]:
# 1hot "x" "o" and "r"
xor_dict = {"x": [1, 0, 0, 0], "o": [0, 1, 0, 0], "r": [0, 0, 1, 0], "v": [0, 0, 0, 1]}

for d in dataset_dict:
    # leave years alone, 1-hot months
    year, month = d.pop('date_review').split("-")[:2]
    d['year_review'] = int(year)
    d['one_hot_months'] = [0] * 12
    d['one_hot_months'][int(month) - 1] = 1
    
    d['recommend'] = xor_dict[d['recommend']]
    d['ceo_approv'] = xor_dict[d['ceo_approv']]
    d['outlook'] = xor_dict[d['outlook']]

{'firm': 'AFH-Wealth-Management',
 'job_title': 'Office Administrator',
 'current': 'Current Employee',
 'overall_rating': 2,
 'work_life_balance': 3,
 'culture_values': 1,
 'career_opp': 2,
 'comp_benefits': 1,
 'senior_mgmt': 4,
 'recommend': [1, 0, 0, 0],
 'ceo_approv': [0, 1, 0, 0],
 'outlook': [0, 0, 1, 0],
 'headline': 'Excellent staff, poor salary',
 'pros': 'Friendly, helpful and hard-working colleagues',
 'cons': "Poor salary which doesn't improve much with progression, no incentive to work harder, high turnover of staff, poor systems",
 'duration': 'more than 1 year',
 'year_review': 2015,
 'one_hot_months': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}